In [ ]:
# Flow3D cavity flow data
import numpy as np
import xarray
from sklearn.metrics import r2_score
import pandas as pd

####################cavity flow####################
Nx, Ny, Nt = 600, 400, 150
dx, dy, dt = 0.2/Nx, 0.03/Ny, 7.5E-10
mu = 1.70E-04
rho = 1.20E-03

U = np.load('../dataset/data/cavity_flow/1-Re100/u_3D.npy')
V = np.load('../dataset/data/cavity_flow/1-Re100/v_3D.npy')
W = np.load('../dataset/data/cavity_flow/1-Re100/w_3D.npy')
P = np.load('../dataset/data/cavity_flow/1-Re100/p_3D.npy')

# # Cut out the portion of the data before the cylinder
# xmin = 220
# xmax = 380
# ymin = 20
# ymax = 180

xmin = 220
xmax = 380
ymin = 20
ymax = 180

####################parse data####################
W = np.transpose(W, axes=[2, 1, 0])
U = np.transpose(U, axes=[2, 1, 0])
V = np.transpose(V, axes=[2, 1, 0])
P = np.transpose(P, axes=[2, 1, 0])

W = W[xmin:xmax,ymin:ymax,:]
U = U[xmin:xmax,ymin:ymax,:]
V = V[xmin:xmax,ymin:ymax,:]
P = P[xmin:xmax,ymin:ymax,:]
Nx, Ny, Nt = W.shape

ds = xarray.Dataset(
    {
        'U': (('x', 'y', 'time'), U),
        'V': (('x', 'y', 'time'), V),
        'W': (('x', 'y', 'time'), W),
        'P': (('x', 'y', 'time'), P),
    },
    coords={
        'x': np.arange(Nx) * dx,
        'y': np.arange(Ny) * dy,
        'time': np.arange(Nt) * dt,
    }
)

def vorticity(ds):
    return (ds.v.differentiate('x') - ds.u.differentiate('y')).rename('vorticity')

W = ds.W
wt = W.differentiate('time')
wx = W.differentiate('x')
wxx = wx.differentiate('x')
wy = W.differentiate('y')
wyy = wy.differentiate('y')

ut = ds.U.differentiate('time')
ux = ds.U.differentiate('x')
uxx = ux.differentiate('x')
uy = ds.U.differentiate('y')
uyy = uy.differentiate('y')

px = ds.P.differentiate('x')
py = ds.P.differentiate('y')

u = np.array(ds.U).reshape(-1,)
v = np.array(ds.V).reshape(-1,)
p = np.array(ds.P).reshape(-1,)
wt = np.array(wt).reshape(-1,)
wx = np.array(wx).reshape(-1,)
wy = np.array(wy).reshape(-1,)
wxx = np.array(wxx).reshape(-1,)
wyy = np.array(wyy).reshape(-1,)

ut = np.array(ut).reshape(-1,)
ux = np.array(ux).reshape(-1,)
uxx = np.array(uxx).reshape(-1,)
uy = np.array(uy).reshape(-1,)
uyy = np.array(uyy).reshape(-1,)

px = np.array(px).reshape(-1,)
py = np.array(py).reshape(-1,)

#############vorticity equation#############
X_library = np.stack(
    (
        u*wx,
        v*wy,
        wxx,
        wyy,
    ),
    axis=-1)
y_library = wt.reshape(-1, 1)

# df_all = pd.DataFrame(np.concatenate([X_library, y_library], axis=1))
# print(df_all.describe())

pred_best = -X_library[:, 0] \
            - X_library[:, 1] \
            + mu*(X_library[:, 2] + X_library[:, 3])

r2 = r2_score(y_library, pred_best.reshape(-1,1))
print('Vorticity: r2 score: ', round(r2, 6))

#############NS equation#############
X_library = np.stack(
    (
        u*ux,
        v*uy,
        px,
        uxx,
        uyy,
    ),
    axis=-1)
y_library = ut.reshape(-1, 1)

# df_all = pd.DataFrame(np.concatenate([X_library, y_library], axis=1))
# print(df_all.describe())

pred_best = -X_library[:, 0] \
            - X_library[:, 1] \
            - 1/rho * X_library[:, 2] \
            + 1/rho * mu * (X_library[:, 3] + X_library[:, 4])

r2 = r2_score(y_library, pred_best.reshape(-1,1))
print('NS-general (x direction): r2 score: ', round(r2, 6))


#############Euler equation#############
X_library = np.stack(
    (
        u*ux,
        v*uy,
        px,
    ),
    axis=-1)
y_library = ut.reshape(-1, 1)

# df_all = pd.DataFrame(np.concatenate([X_library, y_library], axis=1))
# print(df_all.describe())

pred_best = -X_library[:, 0] \
            - X_library[:, 1] \
            - 1/rho * X_library[:, 2]

r2 = r2_score(y_library, pred_best.reshape(-1,1))
print('Euler: r2 score: ', round(r2, 6))


In [1]:
import numpy as np
import xarray
from sklearn.metrics import r2_score
import pandas as pd

####################cavity flow####################
Nx, Ny, Nt = 201, 201, 50000
dx, dy, dt = 1./(Nx-1), 1./(Ny-1), 0.00001
mu = 0.1
rho = 1

U = np.load('../dataset/data/cavity_flow/test/u_3D.npy')
V = np.load('../dataset/data/cavity_flow/test/v_3D.npy')
P = np.load('../dataset/data/cavity_flow/test/p_3D.npy')

# # Cut out the portion of the data before the cylinder
# xmin = 220
# xmax = 380
# ymin = 20
# ymax = 180

xmin = 20
xmax = 180
ymin = 20
ymax = 180

####################parse data####################

U = U[xmin:xmax,ymin:ymax, -200:]
V = V[xmin:xmax,ymin:ymax, -200:]
P = P[xmin:xmax,ymin:ymax, -200:]
Nx, Ny, Nt = U.shape

print('Build ds')
ds = xarray.Dataset(
    {
        'U': (('x', 'y', 'time'), U),
        'V': (('x', 'y', 'time'), V),
        'P': (('x', 'y', 'time'), P),
    },
    coords={
        'x': np.arange(Nx) * dx,
        'y': np.arange(Ny) * dy,
        'time': np.arange(Nt) * dt,
    }
)

print('Calculate derivatives')
ut = ds.U.differentiate('time')
ux = ds.U.differentiate('x')
uxx = ux.differentiate('x')
uy = ds.U.differentiate('y')
uyy = uy.differentiate('y')

px = ds.P.differentiate('x')
py = ds.P.differentiate('y')

u = np.array(ds.U).reshape(-1,)
v = np.array(ds.V).reshape(-1,)
p = np.array(ds.P).reshape(-1,)

ut = np.array(ut).reshape(-1,)
ux = np.array(ux).reshape(-1,)
uxx = np.array(uxx).reshape(-1,)
uy = np.array(uy).reshape(-1,)
uyy = np.array(uyy).reshape(-1,)

px = np.array(px).reshape(-1,)
py = np.array(py).reshape(-1,)

print('Build library and fit')
#############NS equation#############
X_library = np.stack(
    (
        u*ux,
        v*uy,
        px,
        uxx,
        uyy,
    ),
    axis=-1)
y_library = ut.reshape(-1, 1)

df_all = pd.DataFrame(np.concatenate([X_library, y_library], axis=1))
print(df_all.describe())

pred_best = -X_library[:, 0] \
            - X_library[:, 1] \
            - 1/rho * X_library[:, 2] \
            + 1/rho * mu * (X_library[:, 3] + X_library[:, 4])

r2 = r2_score(y_library, pred_best.reshape(-1,1))
print('NS-general (x direction): r2 score: ', round(r2, 6))

Build ds
Calculate derivatives
Build library and fit
NS-general (x direction): r2 score:  -1473.31277


In [1]:
# flow3D cylinder data
import numpy as np
import xarray
from sklearn.metrics import r2_score
import pandas as pd

####################cylinder flow####################
Nx, Ny, Nt = 500, 222, 375
dx, dy, dt = 0.9/Nx, 0.4/Ny, 0.000080
mu = 0.02
W = np.load('../dataset/data/1-Re100-cylinder/w_3D.npy')
U = np.load('../dataset/data/1-Re100-cylinder/u_3D.npy')
V = np.load('../dataset/data/1-Re100-cylinder/v_3D.npy')

xmin = 100
xmax = 425
ymin = 15
ymax = 185

####################parse data####################
W = np.transpose(W, axes=[2, 1, 0])
U = np.transpose(U, axes=[2, 1, 0])
V = np.transpose(V, axes=[2, 1, 0])

# W = np.flip(W, 1)
# U = np.flip(U, 1)
# V = np.flip(V, 1)

W = W[xmin:xmax,ymin:ymax,:]
U = U[xmin:xmax,ymin:ymax,:]
V = V[xmin:xmax,ymin:ymax,:]
Nx, Ny, Nt = W.shape

ds = xarray.Dataset(
    {
        'U': (('x', 'y', 'time'), U),
        'V': (('x', 'y', 'time'), V),
        'W': (('x', 'y', 'time'), W),
    },
    coords={
        'x': np.arange(Nx) * dx,
        'y': np.arange(Ny) * dy,
        'time': np.arange(Nt) * dt,
    }
)

def vorticity(ds):
    return (ds.v.differentiate('x') - ds.u.differentiate('y')).rename('vorticity')

W = ds.W
wt = W.differentiate('time')
wx = W.differentiate('x')
wxx = wx.differentiate('x')
wy = W.differentiate('y')
wyy = wy.differentiate('y')

u = np.array(ds.U).reshape(-1,)
v = np.array(ds.V).reshape(-1,)
wt = np.array(wt).reshape(-1,)
wx = np.array(wx).reshape(-1,)
wy = np.array(wy).reshape(-1,)
wxx = np.array(wxx).reshape(-1,)
wyy = np.array(wyy).reshape(-1,)

#############vorticity equation#############
X_library = np.stack(
    (
        u*wx,
        v*wy,
        wxx,
        wyy,
    ),
    axis=-1)
y_library = wt.reshape(-1, 1)

# df_all = pd.DataFrame(np.concatenate([X_library, y_library], axis=1))
# print(df_all.describe())

pred_best = -X_library[:, 0] \
            - X_library[:, 1] \
            + mu*(X_library[:, 2] + X_library[:, 3])

r2 = r2_score(y_library, pred_best.reshape(-1,1))
print('Vorticity: r2 score: ', round(r2, 6))

Vorticity: r2 score:  0.943896


In [5]:
# Flow3D cylinder flow data
import numpy as np
import xarray
from sklearn.metrics import r2_score
import pandas as pd

####################cavity flow####################
Nx, Ny, Nt = 500, 222, 201
dx, dy, dt = 0.9/Nx, 0.4/Ny, 0.00004
mu = 1.00E-04
rho = 1.11E-03
g = 980.665

l_ref = 1.00E-01
v_ref = 9.00E+01
t_ref = l_ref / v_ref
w_ref = 1 / t_ref
p_ref = 1.00E+05

W = np.load('../dataset/data/13-Re99-cylinder/w_3D.npy')
U = np.load('../dataset/data/13-Re99-cylinder/u_3D.npy')
V = np.load('../dataset/data/13-Re99-cylinder/v_3D.npy')
P = np.load('../dataset/data/13-Re99-cylinder/p_3D.npy')

W = W / w_ref
U = U / v_ref
V = V / v_ref
P = P / p_ref
dx, dy, dt = dx/l_ref, dy/l_ref, dt/t_ref

import math
Re = v_ref * l_ref * rho / mu
Fr = v_ref / math.sqrt(g*l_ref)
Eu = p_ref / rho / v_ref**2
print('Re', Re, 'Fr', Fr, 'Eu', Eu)

# # Cut out the portion of the data before the cylinder
# xmin = 220
# xmax = 380
# ymin = 20
# ymax = 180

xmin = 100
xmax = 425
ymin = 15
ymax = 185

####################parse data####################
W = np.transpose(W, axes=[2, 1, 0])
U = np.transpose(U, axes=[2, 1, 0])
V = np.transpose(V, axes=[2, 1, 0])
P = np.transpose(P, axes=[2, 1, 0])

W = W[xmin:xmax,ymin:ymax, :]
U = U[xmin:xmax,ymin:ymax, :]
V = V[xmin:xmax,ymin:ymax, :]
P = P[xmin:xmax,ymin:ymax, :]
Nx, Ny, Nt = W.shape

ds = xarray.Dataset(
    {
        'U': (('x', 'y', 'time'), U),
        'V': (('x', 'y', 'time'), V),
        'W': (('x', 'y', 'time'), W),
        'P': (('x', 'y', 'time'), P),
    },
    coords={
        'x': np.arange(Nx) * dx,
        'y': np.arange(Ny) * dy,
        'time': np.arange(Nt) * dt,
    }
)

def vorticity(ds):
    return (ds.v.differentiate('x') - ds.u.differentiate('y')).rename('vorticity')

W = ds.W
wt = W.differentiate('time')
wx = W.differentiate('x')
wxx = wx.differentiate('x')
wy = W.differentiate('y')
wyy = wy.differentiate('y')

ut = ds.U.differentiate('time')
ux = ds.U.differentiate('x')
uxx = ux.differentiate('x')
uy = ds.U.differentiate('y')
uyy = uy.differentiate('y')

vt = ds.V.differentiate('time')
vx = ds.V.differentiate('x')
vxx = vx.differentiate('x')
vy = ds.V.differentiate('y')
vyy = vy.differentiate('y')

px = ds.P.differentiate('x')
py = ds.P.differentiate('y')

u = np.array(ds.U).reshape(-1,)
v = np.array(ds.V).reshape(-1,)
p = np.array(ds.P).reshape(-1,)
wt = np.array(wt).reshape(-1,)
wx = np.array(wx).reshape(-1,)
wy = np.array(wy).reshape(-1,)
wxx = np.array(wxx).reshape(-1,)
wyy = np.array(wyy).reshape(-1,)

ut = np.array(ut).reshape(-1,)
ux = np.array(ux).reshape(-1,)
uxx = np.array(uxx).reshape(-1,)
uy = np.array(uy).reshape(-1,)
uyy = np.array(uyy).reshape(-1,)

vt = np.array(vt).reshape(-1,)
vx = np.array(vx).reshape(-1,)
vxx = np.array(vxx).reshape(-1,)
vy = np.array(vy).reshape(-1,)
vyy = np.array(vyy).reshape(-1,)

px = np.array(px).reshape(-1,)
py = np.array(py).reshape(-1,)

rho_g = np.ones_like(vt) * g * rho

#############vorticity equation#############
X_library = np.stack(
    (
        u*wx,
        v*wy,
        wxx,
        wyy,
    ),
    axis=-1)
y_library = wt.reshape(-1, 1)

# df_all = pd.DataFrame(np.concatenate([X_library, y_library], axis=1))
# print(df_all.describe())

pred_best = -X_library[:, 0] \
            - X_library[:, 1] \
            + mu*(X_library[:, 2] + X_library[:, 3])

r2 = r2_score(y_library, pred_best.reshape(-1,1))
print('Vorticity: r2 score: ', round(r2, 6))

#############NS equation (x direction)#############
X_library = np.stack(
    (
        u*ux,
        v*uy,
        uxx,
        uyy,
        px,
    ),
    axis=-1)
y_library = ut.reshape(-1, 1)

df_all = pd.DataFrame(np.concatenate([X_library, y_library], axis=1))
print(df_all.describe())

pred_best = -X_library[:, 0] \
            - X_library[:, 1] \
            + 1/Re * (X_library[:, 2] + X_library[:, 3]) \
            - Eu * X_library[:, 4]

r2 = r2_score(y_library, pred_best.reshape(-1,1))
print('NS-general (x direction): r2 score: ', round(r2, 6))

# #############NS equation (y direction)#############
# X_library = np.stack(
#     (
#         u*vx,
#         v*vy,
#         py,
#         vxx,
#         vyy,
#         rho_g,
#     ),
#     axis=-1)
# y_library = vt.reshape(-1, 1)

# pred_best = -X_library[:, 0] \
#             - X_library[:, 1] \
#             - 1/rho * X_library[:, 2] \
#             + 1/rho * mu * (X_library[:, 3] + X_library[:, 4]) \
#             - 1/rho * X_library[:, 5]

# r2 = r2_score(y_library, pred_best.reshape(-1,1))
# print('NS-general (y direction): r2 score: ', round(r2, 6))

Re 99.9 Fr 9.088289896961484 Eu 11122.233344455566
Vorticity: r2 score:  0.934067
                  0             1             2             3             4  \
count  1.110525e+07  1.110525e+07  1.110525e+07  1.110525e+07  1.110525e+07   
mean  -3.205662e-02 -4.399339e-02  5.126651e-03 -1.886569e-01  6.228426e-06   
std    3.069608e-01  2.780830e-01  6.770344e-01  5.076487e+00  1.821637e-05   
min   -9.774389e-01 -1.448646e+00 -2.072274e+00 -2.822416e+01 -4.777778e-05   
25%   -2.488198e-01 -1.173903e-01 -4.021262e-01 -2.150417e+00 -5.555556e-06   
50%   -1.406744e-03 -3.736650e-04  1.138546e-02  2.662106e-01  5.555556e-06   
75%    1.449414e-01  6.789158e-02  3.755401e-01  2.249241e+00  1.722222e-05   
max    9.488315e-01  9.222145e-01  3.615458e+00  2.285165e+01  8.250000e-05   

                  5  
count  1.110525e+07  
mean   4.913644e-03  
std    2.825643e-01  
min   -9.702605e-01  
25%   -1.468580e-01  
50%    7.179321e-03  
75%    1.771009e-01  
max    1.058007e+00  
NS-gener